In [ ]:
import shutil

import random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import cv2
import csv
import os
import shutil
#import keras.utils as image
import tensorflow as tf
import numpy as np
import pandas as pd
import os
from os import listdir
from PIL import Image
from numpy import array
from numpy import asarray
import seaborn as sns

from tensorflow.python.keras.layers import Dense



from keras import Sequential
from keras.layers import Dense,Flatten,Conv2D,MaxPooling2D
from tensorflow.keras.utils import load_img, img_to_array
from sklearn.model_selection import KFold
from numpy import asarray
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Load the train and test image IDs from separate CSV files
#train_ids = pd.read_csv('/content/drive/MyDrive/AAI_project_1/Pothole/train_ids_labels.csv')
#test_ids = pd.read_csv('/content/drive/MyDrive/AAI_project_1/Pothole/test_ids_only.csv')
test_ids = '/content/drive/MyDrive/AAI_project_1/Pothole/test_ids_only.csv'

# Set paths to the train and test image directories
all_data_path = '/content/drive/MyDrive/AAI_project_1/Pothole/all_data'
train_ids_path = '/content/drive/MyDrive/AAI_project_1/Pothole/train_ids_labels.csv'

new_train_path = '/content/drive/MyDrive/AAI_project_1/Pothole/train'

In [ ]:
train_dir = '/content/drive/MyDrive/AAI_project_1/Pothole/train'
test_dir = '/content/drive/MyDrive/AAI_project_1/Pothole/test'

train_no_pothole_dir = '/content/drive/MyDrive/AAI_project_1/Pothole/train/No_Potholes'
train_yes_pothole_dir = '/content/drive/MyDrive/AAI_project_1/Pothole/train/Potholes'

In [ ]:
print('total training no pothole images :', len(os.listdir(train_no_pothole_dir)))
print('total training pothole images :', len(os.listdir(train_yes_pothole_dir)))

total training no pothole images : 2365
total training pothole images : 1661


In [ ]:
train_size, validation_size, test_size = 420,140,140
img_width, img_height = 224, 224


In [ ]:
from keras.applications import VGG16

# Input Shape: (224 X 224 X 3) where 3 is the number of input color channels, RGB
conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(img_width, img_height, 3))


58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
# Check architecture
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
datagen = ImageDataGenerator(rescale=1./255)
batch_size = 4

def extract_features(directory, sample_count):
    features = np.zeros(shape=(sample_count, 7, 7, 512))  # Must be equal to the output of the convolutional base
    labels = np.zeros(shape=(sample_count))
    # Preprocess data

    generator = datagen.flow_from_directory(directory,
                                            target_size=(img_width,img_height),
                                            batch_size = batch_size,
                                            class_mode='binary')

      #tf.keras.utils.image_dataset_from_directory


    # Pass data through convolutional base
    i = 0
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features[i * batch_size: (i + 1) * batch_size] = features_batch
        labels[i * batch_size: (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            break
    return features, labels

train_features, train_labels = extract_features(train_dir, train_size)  # Agree with our small dataset size
#validation_features, validation_labels = extract_features(validation_dir, validation_size)
#test_features, test_labels = extract_features(test_dir, test_size)

Found 4026 images belonging to 2 classes.
1/1 [==============================] - 2s 2s/step


In [ ]:
print(train_features)
print(train_labels)

[[[[3.02855372e-02 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
    6.34583950e-01 0.00000000e+00]
   [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
    6.25393033e-01 0.00000000e+00]
   [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
    6.64363980e-01 0.00000000e+00]
   ...
   [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
    6.56450510e-01 0.00000000e+00]
   [1.64283618e-01 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
    5.58088064e-01 0.00000000e+00]
   [1.20572418e-01 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
    5.27520895e-01 0.00000000e+00]]

  [[1.28330231e-01 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
    4.61101294e-01 0.00000000e+00]
   [3.23094428e-02 0.00000000e+00 6.13032579e-02 ... 0.00000000e+00
    2.06740856e-01 0.00000000e+00]
   [0.00000000e+00 0.00000000e+00 2.58735418e-02 ... 0.00000000e+00
    3.85171533e-01 0.00000000e+00]
   ...
   [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00

In [ ]:
train_features.shape

(420, 7, 7, 512)

In [ ]:
# Define model
from keras import models
from keras import layers
from keras import optimizers

epochs = 20

model = models.Sequential()
model.add(layers.Flatten(input_shape=(7,7,512)))
model.add(layers.Dense(256, activation='relu', input_dim=(7*7*512)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 256)               6422784   
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 6,423,041
Trainable params: 6,423,041
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Compile model
model.compile(optimizer=optimizers.Adam(),
              loss='binary_crossentropy',
              metrics=['acc'])

In [ ]:
# Train model
history = model.fit(train_features, train_labels,
                    epochs=epochs,
                    batch_size=batch_size,
                    #validation_data=(validation_features, validation_labels)
                     )

Epoch 1/20
105/105 [==============================] - 3s 22ms/step - loss: 1.0629 - acc: 0.6976
Epoch 2/20
105/105 [==============================] - 3s 25ms/step - loss: 0.3873 - acc: 0.8310
Epoch 3/20
105/105 [==============================] - 2s 20ms/step - loss: 0.3602 - acc: 0.8476
Epoch 4/20
105/105 [==============================] - 2s 19ms/step - loss: 0.2923 - acc: 0.8690
Epoch 5/20
105/105 [==============================] - 2s 19ms/step - loss: 0.2455 - acc: 0.8952
Epoch 6/20
105/105 [==============================] - 2s 19ms/step - loss: 0.1763 - acc: 0.9452
Epoch 7/20
105/105 [==============================] - 2s 22ms/step - loss: 0.1872 - acc: 0.9167
Epoch 8/20
105/105 [==============================] - 2s 24ms/step - loss: 0.1531 - acc: 0.9571
Epoch 9/20
105/105 [==============================] - 2s 19ms/step - loss: 0.1521 - acc: 0.9405
Epoch 10/20
105/105 [==============================] - 2s 19ms/step - loss: 0.1346 - acc: 0.9500
Epoch 11/20
105/105 [==================

In [ ]:
#import csv

#prediction = [1, 2, 3, 4, 5]

#with open('/content/drive/MyDrive/AAI_project_1/Pothole/Submission.csv', mode="w", newline="") as csvfile:
#    writer = csv.writer(csvfile)


# Define function for predictions
    #dest_path = os.path.join(new_train_path, image_id + '.JPG')
import csv
test_ids = '/content/drive/MyDrive/AAI_project_1/Pothole/test_ids_only.csv'
'''
def image_predictions(classifier, n_cases):
    for i in range(1,n_cases):
        path = test_dir

'''
# read the CSV file and get the Image_IDs
with open(test_ids, 'r') as k:
    reader2 = csv.DictReader(k)
    my_test_ids = [row['Image_ID'] for row in reader2]

#def image_predictions(classifier, image_ids):
    #for i in range(1,n_cases):
      #  path = test_dir
dataset = pd.read_csv(test_ids)
predictions = []
for my_image_id in my_test_ids:
      my_path = os.path.join(test_dir, my_image_id + '.JPG')

        # Get picture
        #random_img = os.listdir(path)
      #img_path = os.path.join(path) #, random_img
        #img = image.load_img(img_path, target_size=(img_width, img_height))
        #img_tensor = image.img_to_array(img)  # Image data encoded as integers in the 0–255 range
        #img_tensor /= 255.  # Normalize to [0,1] for plt.imshow application

      img = tf.keras.utils.load_img(my_path, target_size=(img_width, img_height))
      img_tensor = tf.keras.utils.img_to_array(img)
      img_tensor /= 255.
        # Extract features
      features = conv_base.predict(img_tensor.reshape(1,img_width, img_height, 3))
      dataset = features
        # Make prediction
      try:
        my_predictions = (model.predict(dataset))
        predictions = my_predictions.append()
        my_prediction = pd.DataFrame(predictions)


        print(my_prediction)

        '''
        for my_image_id in my_test_ids:
          predictions.append(predictions)

        with open('/content/drive/MyDrive/AAI_project_1/Pothole/Submission.csv', mode="w", newline="") as file:
          writer = csv.writer(file)
          writer.writerow(['target'])
          for prediction in predictions:
            writer.writerrow([prediction])

        #prediction.to_csv('/content/drive/MyDrive/AAI_project_1/Pothole/Submission.csv')
        #writer.writerow(prediction)
        '''
      except:
        prediction = [model.predict(features.reshape(1, 7*7*512))].append(prediction)



Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 24ms/step
          0
0  0.999993
1/1 [==============================] - 0s 21ms/step
          0
0  0.987816
1/1 [==============================] - 0s 34ms/step
          0
0  0.999999
1/1 [==============================] - 0s 30ms/step
          0
0  0.999994
1/1 [==============================] - 0s 29ms/step
          0
0  0.841869
1/1 [==============================] - 0s 28ms/step
          0
0  0.963913
1/1 [==============================] - 0s 27ms/step
          0
0  0.537186
1/1 [==============================] - 0s 23ms/step
     0
0  1.0
1/1 [==============================] - 0s 24ms/step
          0
0  0.997862
1/1 [==============================] - 0s 23ms/step
          0
0  0.753074
1/1 [==============================] - 0s 21ms/step
          0
0  0.053847
1/1 [==============================] - 0s 26ms/step
          0
0  0.119058
1/1 [==============================] - 0s 22ms/

In [ ]:
my_prediction.to_csv('/content/drive/MyDrive/AAI_project_1/Pothole/Submission.csv')